In [143]:
import os
import openai
from gitdb.util import write
from sympy import content
!pip install gitpython


In [144]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [145]:
from git import Repo

In [146]:
from pathlib import Path

In [147]:
pwd

'H:\\02_AIWork\\GitHub\\Data'

In [148]:
PATH_TO_BLOG = Path("H:\\02_AIWork\\GitHub\\Data")

In [149]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [150]:
PATH_TO_CONTENT = PATH_TO_BLOG/"content"

In [151]:
PATH_TO_CONTENT

WindowsPath('H:/02_AIWork/GitHub/Data/content')

In [152]:
PATH_TO_CONTENT.mkdir(exist_ok=True, parents=True)

In [153]:
def update_blog(commit_message='Updates blog'):
    # Открываем репозиторий Git
    repo = Repo(PATH_TO_BLOG)
    # git add
    repo.git.add(A=True) # или repo.git.add(".")
    # Создаем коммит
    repo.index.commit(commit_message)
    # Отправляем изменения в удаленный репозиторий
    origin = repo.remote(name='origin')
    origin.push()

In [154]:
random_text_string = "jakdfdjfkljasfljdjflsdajf"

In [155]:
with open(PATH_TO_BLOG/"index.html", "w") as f:
    f.write(random_text_string)

In [156]:
update_blog()

In [174]:
import shutil
def create_new_blog(title,content,cover_image):
    cover_image = Path(cover_image)
    
    # Проверяем, есть ли файлы .html
    existing_files = list(PATH_TO_CONTENT.glob("*.html"))
    
    files = len(existing_files)  # Количество существующих блогов
    new_title = f"{files+1}.html"
    path_to_new_content = PATH_TO_CONTENT / new_title
    
    shutil.copy(cover_image, PATH_TO_CONTENT)
    
    if not os.path.exists(path_to_new_content):
        # WHITE A NEW HTML FILE
        with open(path_to_new_content, "w") as f:
            f.write('<!DOCTYPE html>\n')
            f.write('<html>\n')
            f.write('<head>\n')
            f.write('<meta charset="utf-8">\n')
            f.write('<title>' + title + '</title>\n')
            f.write('</head>\n')
            
            f.write('<body>\n')
            f.write(f"<img src='{cover_image.name}' alt='Cover Image'> <br />\n")
            f.write(f"<h1> {title} </h1>\n")
            # OpenAI --> Completion GPT --> "hello\nblog post\n"
            f.write(content.replace("\n", "<br />\n"))
            f.write('</body>\n')
            f.write('</html>\n')
            print("Blog Created")
            return path_to_new_content
            
    else:
        raise FileNotFoundError("File already exists, please check again you name! Aborting!")
    

In [158]:
path_to_new_content = create_new_blog('Test_title','skdajfldjslkjdsf','example_download.png')

Blog Created


In [159]:
#index.html ---> Blog post

In [160]:
from bs4 import BeautifulSoup as Soup

In [161]:
# pip install BeautifulSoup4

In [162]:
with open(PATH_TO_BLOG/"index.html") as index:
    soup = Soup(index.read())

In [163]:
str(soup)

'jakdfdjfkljasfljdjflsdajf'

In [164]:
# checking duplicate links

# write blog post link ---> index.html

In [171]:
def check_for_duplicate_links(path_to_new_content, links):
    if not links:  # Если `links` пуст, то нет дубликатов
        return False

    urls = [str(link.get("href")) for link in links]  # Теперь эта строка безопасна
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    
    return content_path in urls


In [172]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/'index.html') as index:
        soup = Soup(index.read())
        
    links = soup.find_all("a")    
    
    # Проверяем, есть ли ссылки
    if not links:
        raise ValueError("No links found in index.html. Cannot insert new blog post.")
    
    last_link = links[-1]  # Теперь эта строка безопасна

    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError("Link already exists")
    
    links_to_new_blog = soup.new_tag("a",href=Path(*path_to_new_content.parts[-2:]))
    links_to_new_blog.string = path_to_new_content.name.split('.')[0]
    last_link.insert_after(links_to_new_blog)
    
    with open(PATH_TO_BLOG/'index.html', 'w') as f:
        f.write(str(soup.prettify(formatter="html")))

In [173]:
write_to_index(path_to_new_content)

ValueError: No links found in index.html. Cannot insert new blog post.

In [63]:
update_blog()